# Answer the Initial Questionaire

In [1]:
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.schema.messages import SystemMessage
import yaml
import os
import json
import pandas as pd

with open('config.yaml', 'r') as file:
    config = yaml.safe_load(file)
os.environ['OPENAI_API_KEY'] = config['openai_api_key']

In [2]:
with open("outputs/3_questionnaire.md", "r") as f:
    questionnaire = f.read()

## Have Each Persona Answer the Initial Questionnaire

In [6]:
import glob

files = glob.glob('outputs/6_final_personas/*.md')
for file in files:
    with open(file) as f:
        persona_content = f.read()
    
    print(os.path.basename(file).replace('.md',''))

    chat_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content=(f"""{persona_content}
                     
                     {open('./persona_descriptions/have_each_persona_answer_the_initial_questionnaire.txt').read()}
                     """)
        ),
        HumanMessagePromptTemplate.from_template(
            """Can you to the best of your ability answer this questionare giving very detailed responses: {questionnaire}"""
            ),
    ]
)


    llm = ChatOpenAI(model='gpt-4-turbo-preview') ## make this defined ones and used again and again!!
    answered_questionaire = llm.invoke(chat_template.format_messages(questionnaire=questionnaire)).content

    with open(f"outputs/7_answered_questionares/{os.path.basename(file).replace('.md','')}.md", "w") as text_file:
        text_file.write(answered_questionaire)

Ethics and Bias Reduction Experts The Scalability Strategist
Business Analysts and ROI Experts The Strategic Optimizer
IT Infrastructure Architects The Integration Innovator
Business Analysts and ROI Experts The Efficiency Explorer
Machine Learning Engineers The Integration Expert
Cloud Computing and Deployment Specialists Stakeholder Engagement Facilitator The Collaborator
IT Infrastructure Architects The Data Guardian
Industry Veterans The Tech-Savvy Innovator
Data Privacy and Security Specialists Continuous Learning Innovator
Stakeholder Engagement Coordinators The Integrator
Ethics and Bias Reduction Experts The Compliance Conductor
User Experience (UX) Designers UX Designer for Feedback Systems Persona The Feedback Strategist
Stakeholder Engagement Coordinators The Technophile
Machine Learning Engineers The Ethical Innovator
IT Infrastructure Architects The Ethical AI Advocate
Regulatory Compliance Consultants The Techno-Strategist
Machine Learning Engineers The Continuous Learner

## Convert Initial Questionnaire Responses to JSON

In [3]:
import glob

chat_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content=(open('./persona_descriptions/convert_initial_questionnaire_responses_to_JSON.txt').read())
        ),
        HumanMessagePromptTemplate.from_template(
            """Can you convert the following into a JSON object: {answered_questionnaire}"""
            ),
    ]
)

llm = ChatOpenAI(model='gpt-4-turbo-preview') ## make this defined ones and used again and again!!

files = glob.glob('outputs/7_answered_questionares/*.md')
for file in files:
    print(os.path.basename(file).replace('.md',''))
    with open(file, 'r') as text_file:
        answered_questionnaire = text_file.read()


    answered_questionnaire_json = llm.invoke(chat_template.format_messages(
        answered_questionnaire=answered_questionnaire)).content


    try:
        answered_questionnaire_json = json.loads(answered_questionnaire_json)
        with open(f"outputs/7_answered_questionares/{os.path.basename(file).replace('.md','')}.json", 'w') as f:
            json.dump(answered_questionnaire_json, f)
    except:
        print("   - unable to process")
        continue

/Users/adamfletcher/Documents/GitHub/ai_delphi_framework/.venv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


Ethics and Bias Reduction Experts The Scalability Strategist
Business Analysts and ROI Experts The Strategic Optimizer
IT Infrastructure Architects The Integration Innovator
Business Analysts and ROI Experts The Efficiency Explorer
Machine Learning Engineers The Integration Expert
Cloud Computing and Deployment Specialists Stakeholder Engagement Facilitator The Collaborator
   - unable to process
IT Infrastructure Architects The Data Guardian
Industry Veterans The Tech-Savvy Innovator
Data Privacy and Security Specialists Continuous Learning Innovator
Stakeholder Engagement Coordinators The Integrator
Ethics and Bias Reduction Experts The Compliance Conductor
User Experience (UX) Designers UX Designer for Feedback Systems Persona The Feedback Strategist
Stakeholder Engagement Coordinators The Technophile
Machine Learning Engineers The Ethical Innovator
   - unable to process
IT Infrastructure Architects The Ethical AI Advocate
Regulatory Compliance Consultants The Techno-Strategist
Mac

## Reformat JSONs to bundle questions

In [4]:
responses_df = pd.DataFrame()

files = glob.glob('outputs/7_answered_questionares/*.json')
for file in files:
    expert = os.path.basename(file).replace('.json','')
    print(expert)
    with open(file, 'r') as j:
        response = json.loads(j.read())
    tmp_df = pd.DataFrame.from_dict(response['Answers'])
    tmp_df['Expert'] = expert
    responses_df = pd.concat([responses_df, tmp_df])

Ethics and Bias Reduction Experts The Scalability Strategist
Industry Veterans The Seasoned Strategist
Data Privacy and Security Specialists Scalability Architect
Regulatory Compliance Consultants The Governance Guru
IT Infrastructure Architects The Scalability Strategist
Machine Learning Engineers The Continuous Learner
Industry Veterans The Tech-Savvy Innovator
IT Infrastructure Architects The Ethical AI Advocate
IT Infrastructure Architects The Collaborative Facilitator
Ethics and Bias Reduction Experts The Bias Buster
Business Analysts and ROI Experts The Efficiency Explorer
Machine Learning Engineers The Scalability Architect
Cloud Computing and Deployment Specialists Cloud Architect The Strategic Visionary
Data Privacy and Security Specialists Data Privacy Guardian
User Experience (UX) Designers UX Designer for Feedback Systems Persona The Feedback Strategist
Regulatory Compliance Consultants The Techno-Strategist
Stakeholder Engagement Coordinators The Guardian
Data Privacy and 

In [8]:
def dataframe_to_text(df, question):
    text_lines = []
    line = f"""# {question}

"""
    text_lines.append(line)

    for index, row in df.iterrows():
        line = f"""
## {row['Expert']}: 

{row['Response']}
        """
        text_lines.append(line.replace("['", "").replace("']", "").replace('["', '').replace('"]', ''))
    return " ".join(text_lines)


In [9]:
with open("outputs/3_questionnaire.json", "r") as f:
    questionnare = json.loads(f.read())

questionare_df = pd.DataFrame.from_dict(questionnare['Questionare']) #todo: fix this spelling error

for i in questionare_df['Number'].unique():
    question = questionare_df.loc[questionare_df['Number'] == i]['Question'].values[0]
    formatted_question = dataframe_to_text(responses_df.loc[responses_df['Question'] == i], 
                                       question)

    with open(f"outputs/8_formatted_questions/{question}.md", "w") as text_file:
        text_file.write(formatted_question)
